# Texas University Dataset

This notebook collects information from many different txt files of the datasetinto one json that has a chat, question number, entry id, and a grade for every answer.

In [24]:
import re
import os
import json
import numpy as np


In [142]:
# Read the contents of the text file
with open("texas/data/sent/all.txt", "r") as file:
    text = file.read()

with open("texas/data/sent/questions.txt", "r") as file:
    questions_text = file.read()

with open("texas/data/sent/answers.txt", "r") as file:
    answers_text = file.read()

# Extract question numbers and remove "<STOP>" tokens
question_numbers = re.findall(r'^(\d+\.\d+)', text, flags=re.MULTILINE)
cleaned_text = text.replace("<STOP>", "")
non_repeated_questions = re.findall(r'^(\d+\.\d+)', questions_text, flags=re.MULTILINE)

# Print the extracted question numbers
# for number in question_numbers:
#     print(number)

# Save the cleaned text to a new file
# with open("cleaned_answers.txt", "w") as file:
#     file.write(cleaned_text)

In [143]:
cleaned_text = cleaned_text.split('\n')[:-1]

In [144]:
questions = {}
question_blocks = re.split(r'(\d+\.\d+)', questions_text)
for i in range(1, len(question_blocks), 2):
    question_number = question_blocks[i]
    question = question_blocks[i + 1].strip()
    question = question.replace("<STOP>", "")
    questions[question_number] = question
    

In [145]:
answers = {}
answers_blocks = re.split(r'(\d+\.\d+)', answers_text)
for i in range(1, len(answers_blocks), 2):
    answers_number = answers_blocks[i]
    answer = answers_blocks[i + 1].strip()
    answer = answer.replace("<STOP>", "")
    answers[answers_number] = answer

In [146]:
folder_path = "texas/data/all_scores/"  # Replace with the path to your folder
float_questions = sorted([float(x) for x in set(question_numbers)])
grade_files = []
flag = True
for question_number in float_questions:
    if(flag):
        grade_file = os.path.join(folder_path, str(question_number) + ".txt")
        grade_files.append(grade_file)
    else:
        flag = True

    if(question_number == 11.9):
        grade_file = os.path.join(folder_path, '11.10' + ".txt")
        grade_files.append(grade_file)
        flag = False


grade_file = os.path.join(folder_path, '12.10' + ".txt")
grade_files.append(grade_file)

In [147]:
from collections import OrderedDict
grade_files = list(OrderedDict.fromkeys(grade_files))

In [148]:
len(grade_files)

87

In [159]:
unique = []
grades = []
for file_path in grade_files:
    if file_path.endswith(".txt"):
        all_grades = []
        with open(file_path, "r") as file:
            file_content = file.read()
            if file_content:
                lines = file_content.split("\n")
                for line in lines:
                    if line != "":
                        grades.append(line)
                        all_grades.append(line)
        unique.append(len(set(all_grades)))

grades = [float(x) for x in grades]


In [167]:
unique
eval_questions = [non_repeated_questions[index] for index, element in enumerate(unique) if element < 4]
len(eval_questions)


11

In [169]:
data = []
count = 0

prev_question_number = 0
for idx,(number, student_answer) in enumerate(zip(question_numbers, cleaned_text)):
    
    # if prev_question_number != number:
    #     data.append({
    #         'entry_id': count,
    #         'chat': "Human: "+questions[number] + "\n\nAssistant: " + answers[number],
    #         # 'question': questions[number],
    #         # 'answer': answers[number],
    #         'label': "positive"
    #     })
    #     prev_question_number = number
    #     count += 1
    data.append({
        'entry_id': count,
        'question_number': number,
        'chat': "Human: "+questions[number] + "\n\nAssistant: " +  re.sub(r'^\d+\.\d+\s+', '',student_answer, count = 1),
        # 'question':  questions[number],
        # 'answer': re.sub(r'^\d+\.\d+\s+', '',student_answer, count = 1),
        'grade': grades[idx]
    })
    count += 1


In [170]:
with open('texas_data.json', 'w') as file:
    json.dump(data, file, indent=4)

In [215]:
import json
import random

# Read the JSON file and parse its contents
with open('texas_data.json', 'r') as file:
    data = json.load(file)

# Group the entries based on question numbers
groups = {}
for entry in data:
    question_number = entry['question_number']
    if question_number not in groups:
        groups[question_number] = []
    groups[question_number].append(entry)

In [216]:
eval_groups = {key: groups[key] for key in list(groups.keys()) if key in eval_questions}
train_groups = {key: groups[key] for key in list(groups.keys()) if key not in eval_questions}

In [217]:
len(train_groups)

76

In [218]:
sampled_entries = []
unsampled_entries = []

# Iterate over each group and sample 5 unique grades or add all entries if unique grades < 5
for question_number, entries in train_groups.items():
    unique_grades = list(set(entry['grade'] for entry in entries))
    
    if 5.0 in unique_grades:
        unique_grades.remove(5.0)
        sampled_grades = random.sample(unique_grades, min(3, len(unique_grades)))
        sampled_grades.append(5.0)
    else:
        sampled_grades = random.sample(unique_grades, min(4, len(unique_grades)))
    ordered_entries = sorted(entries, key=lambda entry: entry['grade'],  reverse=True)
    sampled_grades = sorted(sampled_grades, reverse=True)

    if len(unique_grades) < 4:
        continue
    
    for entry in ordered_entries:
        if entry['grade'] in sampled_grades:
            sampled_entries.append(entry)
            sampled_grades.remove(entry['grade'])
        else:
            unsampled_entries.append(entry)


# Save the sampled entries in a new JSON file
with open('train_data1.json', 'w') as file:
    json.dump(sampled_entries, file, indent=4)

# Save the unsampled entries in another JSON file
with open('unsampled_data1.json', 'w') as file:
    json.dump(unsampled_entries, file, indent=4)


In [219]:
import json
import random

# Read the JSON file and parse its contents
with open('unsampled_data1.json', 'r') as file:
    unsampled_data = json.load(file)

# Group the entries based on question numbers
groups = {}
for entry in unsampled_data:
    question_number = entry['question_number']
    if question_number not in groups:
        groups[question_number] = []
    groups[question_number].append(entry)

In [220]:
sampled_entries = []
unsampled_entries = []

# Iterate over each group and sample 5 unique grades or add all entries if unique grades < 5
for question_number, entries in groups.items():
    unique_grades = list(set(entry['grade'] for entry in entries))
    
    if 5.0 in unique_grades:
        unique_grades.remove(5.0)
        sampled_grades = random.sample(unique_grades, min(3, len(unique_grades)))
        sampled_grades.append(5.0)
    else:
        sampled_grades = random.sample(unique_grades, min(4, len(unique_grades)))
    ordered_entries = sorted(entries, key=lambda entry: entry['grade'],  reverse=True)
    sampled_grades = sorted(sampled_grades, reverse=True)
    
    if len(unique_grades) < 4:
        continue
    
    for entry in ordered_entries:
        if entry['grade'] in sampled_grades:
            sampled_entries.append(entry)
            sampled_grades.remove(entry['grade'])
        else:
            unsampled_entries.append(entry)
            
        

# Save the sampled entries in a new JSON file
with open('train_data2.json', 'w') as file:
    json.dump(sampled_entries, file, indent=4)

# Save the unsampled entries in another JSON file
with open('unsampled_data2.json', 'w') as file:
    json.dump(unsampled_entries, file, indent=4)
    

In [221]:
len(sampled_entries)

208

In [222]:
import json
import random

# Read the JSON file and parse its contents
with open('unsampled_data2.json', 'r') as file:
    unsampled_data = json.load(file)

# Group the entries based on question numbers
groups = {}
for entry in unsampled_data:
    question_number = entry['question_number']
    if question_number not in groups:
        groups[question_number] = []
    groups[question_number].append(entry)

In [223]:
sampled_entries = []
unsampled_entries = []

# Iterate over each group and sample 5 unique grades or add all entries if unique grades < 5
for question_number, entries in groups.items():
    unique_grades = list(set(entry['grade'] for entry in entries))
    
    if 5.0 in unique_grades:
        unique_grades.remove(5.0)
        sampled_grades = random.sample(unique_grades, min(3, len(unique_grades)))
        sampled_grades.append(5.0)
    else:
        sampled_grades = random.sample(unique_grades, min(4, len(unique_grades)))
    ordered_entries = sorted(entries, key=lambda entry: entry['grade'],  reverse=True)
    sampled_grades = sorted(sampled_grades, reverse=True)
    
    if len(unique_grades) < 4:
        continue
    
    for entry in ordered_entries:
        if entry['grade'] in sampled_grades:
            sampled_entries.append(entry)
            sampled_grades.remove(entry['grade'])
        else:
            unsampled_entries.append(entry)
            
        

# Save the sampled entries in a new JSON file
with open('train_data3.json', 'w') as file:
    json.dump(sampled_entries, file, indent=4)

# Save the unsampled entries in another JSON file
with open('unsampled_data3.json', 'w') as file:
    json.dump(unsampled_entries, file, indent=4)
    

In [224]:
len(sampled_entries)

128

In [225]:
import json
import random

# Read the JSON file and parse its contents
with open('unsampled_data3.json', 'r') as file:
    unsampled_data = json.load(file)

# Group the entries based on question numbers
groups = {}
for entry in unsampled_data:
    question_number = entry['question_number']
    if question_number not in groups:
        groups[question_number] = []
    groups[question_number].append(entry)

In [226]:
sampled_entries = []
unsampled_entries = []

# Iterate over each group and sample 5 unique grades or add all entries if unique grades < 5
for question_number, entries in groups.items():
    unique_grades = list(set(entry['grade'] for entry in entries))
    
    if 5.0 in unique_grades:
        unique_grades.remove(5.0)
        sampled_grades = random.sample(unique_grades, min(3, len(unique_grades)))
        sampled_grades.append(5.0)
    else:
        sampled_grades = random.sample(unique_grades, min(4, len(unique_grades)))
    ordered_entries = sorted(entries, key=lambda entry: entry['grade'],  reverse=True)
    sampled_grades = sorted(sampled_grades, reverse=True)
    
    if len(unique_grades) < 4:
        continue
    
    for entry in ordered_entries:
        if entry['grade'] in sampled_grades:
            sampled_entries.append(entry)
            sampled_grades.remove(entry['grade'])
        else:
            unsampled_entries.append(entry)
            
        

# Save the sampled entries in a new JSON file
with open('train_data4.json', 'w') as file:
    json.dump(sampled_entries, file, indent=4)

# Save the unsampled entries in another JSON file
with open('unsampled_data4.json', 'w') as file:
    json.dump(unsampled_entries, file, indent=4)
    

In [227]:
len(sampled_entries)

88

In [ ]:
import json
import random

# Read the JSON file and parse its contents
with open('unsampled_data4.json', 'r') as file:
    unsampled_data = json.load(file)

# Group the entries based on question numbers
groups = {}
for entry in unsampled_data:
    question_number = entry['question_number']
    if question_number not in groups:
        groups[question_number] = []
    groups[question_number].append(entry)

In [228]:
sampled_entries = []
unsampled_entries = []

# Iterate over each group and sample 5 unique grades or add all entries if unique grades < 5
for question_number, entries in groups.items():
    unique_grades = list(set(entry['grade'] for entry in entries))
    
    if 5.0 in unique_grades:
        unique_grades.remove(5.0)
        sampled_grades = random.sample(unique_grades, min(3, len(unique_grades)))
        sampled_grades.append(5.0)
    else:
        sampled_grades = random.sample(unique_grades, min(4, len(unique_grades)))
    ordered_entries = sorted(entries, key=lambda entry: entry['grade'],  reverse=True)
    sampled_grades = sorted(sampled_grades, reverse=True)
    
    if len(unique_grades) < 4:
        continue
    
    for entry in ordered_entries:
        if entry['grade'] in sampled_grades:
            sampled_entries.append(entry)
            sampled_grades.remove(entry['grade'])
        else:
            unsampled_entries.append(entry)
            
        

# Save the sampled entries in a new JSON file
with open('train_data5.json', 'w') as file:
    json.dump(sampled_entries, file, indent=4)

# Save the unsampled entries in another JSON file
with open('unsampled_data5.json', 'w') as file:
    json.dump(unsampled_entries, file, indent=4)
    

In [229]:
len(sampled_entries)

88

In [230]:
import json
import random

# Read the JSON file and parse its contents
with open('unsampled_data4.json', 'r') as file:
    unsampled_data = json.load(file)

# Group the entries based on question numbers
groups = {}
for entry in unsampled_data:
    question_number = entry['question_number']
    if question_number not in groups:
        groups[question_number] = []
    groups[question_number].append(entry)

In [231]:
sampled_entries = []
unsampled_entries = []

# Iterate over each group and sample 5 unique grades or add all entries if unique grades < 5
for question_number, entries in groups.items():
    unique_grades = list(set(entry['grade'] for entry in entries))
    
    if 5.0 in unique_grades:
        unique_grades.remove(5.0)
        sampled_grades = random.sample(unique_grades, min(3, len(unique_grades)))
        sampled_grades.append(5.0)
    else:
        sampled_grades = random.sample(unique_grades, min(4, len(unique_grades)))
    ordered_entries = sorted(entries, key=lambda entry: entry['grade'],  reverse=True)
    sampled_grades = sorted(sampled_grades, reverse=True)
    
    if len(unique_grades) < 4:
        continue
    
    for entry in ordered_entries:
        if entry['grade'] in sampled_grades:
            sampled_entries.append(entry)
            sampled_grades.remove(entry['grade'])
        else:
            unsampled_entries.append(entry)
            
        

# Save the sampled entries in a new JSON file
with open('train_data6.json', 'w') as file:
    json.dump(sampled_entries, file, indent=4)

# Save the unsampled entries in another JSON file
with open('unsampled_data6.json', 'w') as file:
    json.dump(unsampled_entries, file, indent=4)
    

In [232]:
len(sampled_entries)

56

In [265]:
sampled_entries = []
bad_sampled_entries = []
# Iterate over the dictionary
for question_number, entries in eval_groups.items():
    grade_5_entries = [entry for entry in entries if entry['grade'] == 5.0]
    bad_grades_entries = [entry for entry in entries if entry['grade'] <= 3.5]
    # Sample 4 entries if there are at least 4 grade 5 entries
    if len(grade_5_entries) >= 4:
        sampled_entries.extend(random.sample(grade_5_entries, 4))
    if len(bad_grades_entries) >= 3:
        bad_sampled_entries.extend(random.sample(bad_grades_entries, 4))
        
# Add the label "positive" to the sampled entries
for entry in sampled_entries:
    entry['label'] = 'positive'
for entry in bad_sampled_entries:
    entry['label'] = 'negative'
eval_entries = sampled_entries + bad_sampled_entries
# Save the sampled entries in a JSON file
with open('eval_data.json', 'w') as file:
    json.dump(eval_entries, file, indent=4)

In [264]:
bad_sampled_entries

[{'entry_id': 802,
  'question_number': '4.6',
  'chat': 'Human: Using an index outside the bounds of the array generates an error.  Is this a compilation error or a run-time error? \n\nAssistant: compilation error. ',
  'grade': 1.0,
  'label': 'negative'},
 {'entry_id': 804,
  'question_number': '4.6',
  'chat': 'Human: Using an index outside the bounds of the array generates an error.  Is this a compilation error or a run-time error? \n\nAssistant: not answered ',
  'grade': 0.0,
  'label': 'negative'},
 {'entry_id': 807,
  'question_number': '4.6',
  'chat': 'Human: Using an index outside the bounds of the array generates an error.  Is this a compilation error or a run-time error? \n\nAssistant: compilation error ',
  'grade': 1.0,
  'label': 'negative'}]